In [12]:
#사전 준비
from pathlib import Path
import os

src_dir=str(Path.home())+"/src/"

data_file = Path(src_dir+"cifar10_val.rec")
if data_file.is_file():
    os.chdir(src_dir)
else:
    !aws s3 cp s3://aws-tc-largeobjects/AWS-200-DLG/v1.0/cifar10_val.rec .
    !aws s3 cp s3://aws-tc-largeobjects/AWS-200-DLG/v1.0/cifar10_train.rec .
    !mkdir symbols
    !aws s3 cp s3://us-west-2-tcprod/courses/ILT-TF-200-MLDEEP/v1.1.2/lab-2-cnn/scripts/symbols/ ./symbols/ --recursive



# CIFAR10 데이터 세트에서의 이미지 예측을 위한 합성곱 신경망(CNN) 교육

이번 실습에서는 이미지를 10개의 알려진 카테고리로 분류하기 위해 CIFAR-10 교육 데이터로 ResNet 신경망을 교육합니다. 해당 코드는 MXNet에서 작성됩니다.

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 및 CIFAR-100은 8천만 개의 작은 이미지 데이터 세트의 하위 세트로 분류됩니다. Alex Krizhevsky, Vinod Nair 및 Geoffrey Hinton이 수집한 이미지입니다.

CIFAR-10 데이터 세트는 60,000개의 32x32 컬러 이미지로 구성됩니다. 이미지는 10종류로 분류되며 분류당 6,000개의 이미지가 있습니다. 50,000개의 교육용 이미지와 10,000개의 테스트 이미지가 있습니다.

![cifar-10](https://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.2/lab-2-cnn/instructions/en_us/images/cifar-10.png)

데이터 세트는 다섯 개의 교육 배치와 하나의 테스트 배치로 나뉘며, 배치당 10,000개의 이미지가 있습니다. 테스트 배치에는 각 분류에서 무작위로 선택한 1,000개의 이미지가 들어 있습니다. 교육 배치에는 나머지 이미지가 무작위로 들어 있지만, 일부 교육 패치에는 한 분류 이미지가 다른 분류 이미지보다 더 들어 있을 수 있습니다. 이러한 사항 외에도, 교육 배치에는 각 분류의 이미지가 정확하게 5,000개 들어 있습니다. 다음은 이미지 데이터 세트로 작업하는 일반적인 방법입니다.

- 분류
- 국지화
- 조각화
- 장면 분류
- 하늘, 도로, 사람, 침대와 같은 의미 카테고리와 관련된 여러 이미지 영역으로 이미지를 분할하고 구문 분석하기 위한 [장면 구문 분석](http://sceneparsing.csail.mit.edu/)

이미지 딥 러닝에 대해 자세히 알아보려면 다음 강좌가 유용합니다. [CS231n: Convolutional Neural Networks for Visual Recognition](http://cs231n.stanford.edu/slides/2016/winter1516_lecture8.pdf)

이번 실습에서는 ResNet \(Residual Net\) [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)을 사용합니다.

![](https://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.2/lab-2-cnn/instructions/en_us/images/resnet.png)

- VGG 네트보다 8배 더 깊지만, 복잡성은 더 낮은 최대 152개 층을 가진 Residual Network입니다.
- ImageNet 테스트 세트에서 3.57% 오류율을 달성했습니다.
- 2015년 ILSVRC\(Large Scale Visual Recognition Challenge\) 분류 작업에서 1위를 차지했습니다.

**목표**

본 실습을 완료하면 다음을 할 수 있게 됩니다.

- 합성곱 신경망 모델 생성
- 합성곱 신경망 교육
- 예측 평가
- 정확도를 높이기 위해 하이퍼파라미터\(배치 크기, 학습 속도 등\) 조율

**사전 조건**

본 실습에는 다음 사전 조건이 필요합니다.

- Microsoft Windows, Mac OS X 또는 Linux\(Ubuntu, SuSE, Red Hat\)가 실행되며 Wi-Fi가 되는 노트북 사용
- iPad나 태블릿 디바이스로는 Qwiklabs 실습 환경에 액세스할 수 없지만 수강생 안내서는 볼 수 있습니다.
- Microsoft Windows 사용자의 경우, 컴퓨터의 관리자 권한이 필요합니다.
- Chrome, Firefox 또는 IE9과 같은 인터넷 브라우저\(IE9 이전 버전은 지원 안 됨\).

**소요 시간**

본 실습에는 약 **30분** 정도가 소요됩니다.

---

## 과제 4: 합성곱 신경망 모델 생성 및 교육

다음 코드를 Jupyter 노트북에 붙여넣어 노트북 셀로 종속성을 가져옵니다.

In [1]:
import os, sys
import argparse
import logging
import mxnet as mx
import random
from mxnet.io import DataBatch, DataIter
import numpy as np
import time
import subprocess
import errno

데이터에 대한 하이퍼파라미터를 정의하려면 파서에 인수로 추가합니다.

In [2]:
def add_data_args(parser):
    data = parser.add_argument_group('Data', 'the input images')
    data.add_argument('--data-train', type=str, help='the training data')
    data.add_argument('--data-val', type=str, help='the validation data')
    data.add_argument('--rgb-mean', type=str, default='123.68,116.779,103.939',
                      help='a tuple of size 3 for the mean rgb')
    data.add_argument('--pad-size', type=int, default=0,
                      help='padding the input image')
    data.add_argument('--image-shape', type=str,
                      help='the image shape feed into the network, e.g. (3,224,224)')
    data.add_argument('--num-classes', type=int, help='the number of classes')
    data.add_argument('--num-examples', type=int, help='the number of training examples')
    data.add_argument('--data-nthreads', type=int, default=4,
                      help='number of threads for data decoding')
    return data

이미지 데이터 반복자를 정의합니다.

In [3]:
def get_rec_iter(args, kv=None):
    image_shape = tuple([int(l) for l in args.image_shape.split(',')])
    dtype = np.float32

    if kv:
        (rank, nworker) = (kv.rank, kv.num_workers)
    else:
        (rank, nworker) = (0, 1)

    rgb_mean = [float(i) for i in args.rgb_mean.split(',')]

    train = mx.io.ImageRecordIter(
        path_imgrec         = args.data_train,
        label_width         = 1,
        mean_r              = rgb_mean[0],
        mean_g              = rgb_mean[1],
        mean_b              = rgb_mean[2],
        data_name           = 'data',
        label_name          = 'softmax_label',
        data_shape          = image_shape,
        batch_size          = args.batch_size,
        pad                 = args.pad_size,
        fill_value          = 127,
        preprocess_threads  = args.data_nthreads,
        shuffle             = True,
        num_parts           = nworker,
        part_index          = rank)
    if args.data_val is None:
        return (train, None)
    val = mx.io.ImageRecordIter(
        path_imgrec         = args.data_val,
        label_width         = 1,
        mean_r              = rgb_mean[0],
        mean_g              = rgb_mean[1],
        mean_b              = rgb_mean[2],
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = args.batch_size,
        data_shape          = image_shape,
        preprocess_threads  = args.data_nthreads,
        num_parts           = nworker,
        part_index          = rank)
    return (train, val)

매 epoch 후 모델을 저장하려면, 데이터 세트 교육용 유틸리티 함수 **\_save\_model**을 사용합니다. 새 셀에서 다음을 실행합니다.

In [4]:
def _save_model(args, rank=0):
    if args.model_prefix is None:
        return None

    return mx.callback.do_checkpoint(args.model_prefix if rank == 0 else "%s-%d" % (
        args.model_prefix, rank))

모델을 실행하려면, fit 함수를 사용하여 model.fit을 호출합니다. **add\_fit\_args** 함수는 fit 함수에 필요한 하이퍼파라미터를 추가합니다. 코드 내에서 각 파라미터에 대한 사용자 도움말 섹션이 제공됩니다.

In [5]:
def add_fit_args(parser):
    """
    parser : argparse.ArgumentParser
    return a parser added with args required by fit
    """
    train = parser.add_argument_group('Training', 'model training')
    train.add_argument('--network', type=str,
                       help='the neural network to use')
    train.add_argument('--num-layers', type=int,
                       help='number of layers in the neural network, required by some networks such as resnet')
    train.add_argument('--gpus', type=str,
                       help='list of gpus to run, e.g. 0 or 0,2,5. empty means using cpu')
    train.add_argument('--kv-store', type=str, default='device',
                       help='key-value store type')
    train.add_argument('--num-epochs', type=int, default=100,
                       help='max num of epochs')
    train.add_argument('--lr', type=float, default=0.1,
                       help='initial learning rate')
    train.add_argument('--optimizer', type=str, default='sgd',
                       help='the optimizer type')
    train.add_argument('--mom', type=float, default=0.9,
                       help='momentum for sgd')
    train.add_argument('--wd', type=float, default=0.0001,
                       help='weight decay for sgd')
    train.add_argument('--batch-size', type=int, default=128,
                       help='the batch size')
    train.add_argument('--disp-batches', type=int, default=40,
                       help='show progress for every n batches')
    train.add_argument('--model-prefix', type=str,
                       help='model prefix')
    parser.add_argument('--monitor', dest='monitor', type=int, default=0,
                        help='log network parameters every N iters if larger than 0')
    return train

def fit(args, network, data_loader, **kwargs):
    """
    train a model
    args : argparse returns
    network : the symbol definition of the neural network
    data_loader : function that returns the train and val data iterators
    """
    # kvstore
    kv = mx.kvstore.create(args.kv_store)
    print("args kvstore is %s"%(kv))

    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    logging.basicConfig(level=logging.DEBUG, format=head)
    logging.info('start with arguments %s', args)

    # data iterators
    (train, val) = data_loader(args, kv)



    # save model
    checkpoint = _save_model(args, kv.rank)

    # devices for training
    devs = mx.gpu(0)

    # create model
    model = mx.mod.Module(
        context       = devs,
        symbol        = network
    )

    optimizer_params = {
            'learning_rate': args.lr,
            'momentum' : args.mom,
            'wd' : args.wd}

    monitor = mx.mon.Monitor(args.monitor, pattern=".*") if args.monitor > 0 else None

    if args.network == 'alexnet':
        # AlexNet will not converge using Xavier
        initializer = mx.init.Normal()
    else:
        initializer = mx.init.Xavier(
            rnd_type='gaussian', factor_type="in", magnitude=2)

    # evaluation metrices
    eval_metrics = ['accuracy']

    # callbacks that run after each batch
    batch_end_callbacks = [mx.callback.Speedometer(args.batch_size, args.disp_batches)]

    # run
    model.fit(train,
        num_epoch          = args.num_epochs,
        eval_data          = val,
        eval_metric        = eval_metrics,
        kvstore            = kv,
        optimizer          = args.optimizer,
        optimizer_params   = optimizer_params,
        initializer        = initializer,
        batch_end_callback = batch_end_callbacks,
        epoch_end_callback = checkpoint,
        allow_missing      = True,
        monitor            = monitor)

이 모델에 사용된 네트워크는 **import\_module** 함수를 사용하여 정의됩니다. 이 모드를 위해 Residual Neural Network\(ResNet\)을 사용합니다. Resnet용 코드는: [Residual Neural Network](https://us-west-2-tcprod.s3.amazonaws.com/courses/ILT-TF-200-MLDEEP/v1.1.2/lab-2-cnn/scripts/symbols/resnet.py)모델을 실행하려면 다음을 실행:

In [6]:
logger = logging.getLogger()

if logger.handlers:
    logger.handlers[0].close()
    logger.handlers = []

fhandler = logging.FileHandler(filename='lab2.log', mode='w')
console = logging.StreamHandler()

# tell the handler to use this format
formatter = logging.Formatter('%(asctime)s - %(message)s')
fhandler.setFormatter(formatter)
console.setFormatter(formatter)
# add the handler to the root logger
logger.addHandler(fhandler)
logger.addHandler(console)
console.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

if __name__ == '__main__':
    # download data
    (train_fname, val_fname) = ('./cifar10_train.rec','./cifar10_val.rec')

    # parse args
    parser = argparse.ArgumentParser(description="train cifar10",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    add_fit_args(parser)
    add_data_args(parser)
    parser.set_defaults(
        # network
        network        = 'resnet', #Network Name
        num_layers     = 50, #Number of layers in the network

        # data
        data_train     = train_fname, # Training dataset
        data_val       = val_fname, # Validation data
        num_classes    = 10, # number of classes
        num_examples  = 50000,
        image_shape    = '3,28,28',
        pad_size       = 4,

        # train
        batch_size     = 128,
        num_epochs     = 5,
        lr             = .01
    )
    args = parser.parse_args('--model-prefix model'.split())

    # load network
    print(args.network)
    from importlib import import_module
    net = import_module('symbols.'+args.network)
    sym = net.get_symbol(**vars(args))
    print(sym)

    model_prefix = 'mx_resnet'
    checkpoint = mx.callback.do_checkpoint(model_prefix)
    # train
    fit(args, sym, get_rec_iter, epoch_end_callback=checkpoint)

2018-11-18 14:49:50,653 - start with arguments Namespace(batch_size=128, data_nthreads=4, data_train='./cifar10_train.rec', data_val='./cifar10_val.rec', disp_batches=40, gpus=None, image_shape='3,28,28', kv_store='device', lr=0.01, model_prefix='model', mom=0.9, monitor=0, network='resnet', num_classes=10, num_epochs=5, num_examples=50000, num_layers=50, optimizer='sgd', pad_size=4, rgb_mean='123.68,116.779,103.939', wd=0.0001)


resnet
<Symbol softmax>
args kvstore is <mxnet.kvstore.KVStore object at 0x7f404e1a5d68>


2018-11-18 14:50:45,065 - Epoch[0] Batch [40]	Speed: 1080.18 samples/sec	accuracy=0.169398
2018-11-18 14:50:49,717 - Epoch[0] Batch [80]	Speed: 1100.97 samples/sec	accuracy=0.260742
2018-11-18 14:50:54,378 - Epoch[0] Batch [120]	Speed: 1098.77 samples/sec	accuracy=0.324414
2018-11-18 14:50:59,042 - Epoch[0] Batch [160]	Speed: 1098.25 samples/sec	accuracy=0.357422
2018-11-18 14:51:03,711 - Epoch[0] Batch [200]	Speed: 1097.13 samples/sec	accuracy=0.393750
2018-11-18 14:51:08,377 - Epoch[0] Batch [240]	Speed: 1097.51 samples/sec	accuracy=0.422852
2018-11-18 14:51:13,039 - Epoch[0] Batch [280]	Speed: 1098.72 samples/sec	accuracy=0.445312
2018-11-18 14:51:17,706 - Epoch[0] Batch [320]	Speed: 1097.38 samples/sec	accuracy=0.472852
2018-11-18 14:51:22,364 - Epoch[0] Batch [360]	Speed: 1099.55 samples/sec	accuracy=0.489258
2018-11-18 14:51:25,856 - Epoch[0] Train-accuracy=0.498177
2018-11-18 14:51:25,858 - Epoch[0] Time cost=46.020
2018-11-18 14:51:25,969 - Saved checkpoint to "model-0001.param

**fit** 함수는 CIFAR-10 데이터 세트에서 모델을 교육합니다. epoch마다 교육 및 검증 정확도가 표시됩니다. 이러한 정확도는 **lab2.log** 파일에도 기록됩니다. 교육 정확도 및 검증 정확도 대비 epoch 곡선은 다음 단계에서 구성됩니다. 이러한 곡선은 모델 교육 시 모델 수행 방식을 이해하는 데 도움이 됩니다. 또한, 하이퍼파라미터 튜닝에도 도움이 됩니다.

교육 및 검증 정확도를 구성하려면 다음번 셀에 다음 코드를 붙여넣습니다.

In [7]:
import bokeh
from bokeh.plotting import figure, output_file, show,output_notebook
import re

output_notebook()

with open("lab2.log","r") as f:
    lines = f.readlines()
f.close()
string_lines = " ".join(lines)

train_acc = re.findall(r'.*Epoch\[\d+\] Train.*=([.\d]+)',string_lines)
valid_acc = re.findall(r'.*Epoch\[\d+\] Valid.*=([.\d]+)',string_lines)

p = figure(plot_width=800, plot_height=400,x_axis_label='Number of Epochs',y_axis_label='Accuracy',toolbar_location='above')

x = list(range(1,args.num_epochs+1))

# add a line renderer
p.line(x,train_acc, line_width=2,color='blue',legend='train')
p.circle(x,train_acc, line_width=2,color='blue')
p.line(x,valid_acc, line_width=2,color='red',legend='valid')
p.circle(x,valid_acc, line_width=2,color='red')
p.legend.location = 'bottom_right'
p.xaxis[0].ticker.desired_num_ticks = args.num_epochs
show(p)

Loading BokehJS ...

---
## 과제 5: 하이퍼파라미터 튜닝

다음 파라미터를 변경하고 교육 및 테스트 오류 변화를 관찰합니다.

|설정 |값|
|---|---|
| 계층 수 **num\_layers** | 50\(기본값\), 110, 150 등으로 시도합니다. |
| 학습 속도 **lr** | 0\.1, 0.05, 0.01\(기본값\), 0.005, 0.001 등으로 시도합니다.|
| 배치 크기 **batch\_size** | 64, 128\(기본값\), 256 등으로 시도합니다.|

## \[선택 과제\]Amazon S3에 교육된 모델 업로드

마지막으로 저장된 **model-xxx.params** 및 **model-symbol.json**\(기본\) 모델을 사용하고 교육된 모델을 Amazon S3 버킷에 업로드합니다.